In [ ]:
"""
stage 1:
build person locationCube
---

2D
matrix
schema: personid lat lon timestamp
research web for sample data that matches schema.
found data sample from kaggle:china-mobile-events-talkingdata
status: done

3D
scatter plot = locationCube
axis: lat lon timestamp
plot: personid
status: Work-In-Progress (WiP)

stage 2:
build distanceCube
---

2D
matrix
personid1 personid2 distance timestamp

3D
scatter plot = distanceCube
axis: personid1 personid2 timestamp
plot: distance
transform: 
distance[bool] = distance <= 30;
or
distance[float] = 1 / distance;
status: WiP


stage 3:
build proximityCube
---
scan distanceCube by timestamp
get: max
"""

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import os
import matplotlib.pyplot as plt
import seaborn as sns 
from mpl_toolkits.basemap import Basemap
%matplotlib inline

In [ ]:
# ls /kaggle/input/china-mobile-user-gemographics/

### This is just a small script to plot some of the lat/lon data.

In [ ]:
# !apt-get install unzip

In [ ]:
# !conda install -c conda-forge basemap-data-hires

In [ ]:
# !unzip /kaggle/input/china-mobile-user-gemographics/events.csv.zip

In [ ]:
# !unzip /kaggle/input/china-mobile-user-gemographics/gender_age_train.csv.zip

In [ ]:
# ls -l /kaggle/working/

In [ ]:
#fname = "../input/events.csv"
fname = "/kaggle/input/china-mobile-user-gemographics/events.csv"
df_events = pd.read_csv(fname, dtype={'device_id': np.str})
df_events.head()

In [ ]:
df00 = df_events
df00 = df00.loc[:, 'device_id timestamp longitude latitude'.split(' ')]
df00['ts'] = [ int(dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S').timestamp()) for x in df00['timestamp']]

In [ ]:
df00_uniqueTimestamps = df00['ts'].drop_duplicates()
lmin = df00_uniqueTimestamps.min()
lmax = df00_uniqueTimestamps.max()
lrange = lmax - lmin
print('lrange: %s' % lrange)
print('len df00 index: %s' % len(df00_uniqueTimestamps.index))
df0 = df00[df00['ts'] == lmin] # slice001
df0

In [ ]:
import traceback, re
from math import radians, cos, sin, asin, sqrt

def distanceCube(df):
    # start distanceCube
    df = df[ (df['longitude'] > 0) ]
    #print(df)
    df1 = pd.DataFrame( np.zeros((len(df.index), len(df.index))), index=df['device_id'], columns=df['device_id'])
    for i in df1.index:
        for j in df1.columns:
            lon = [df.loc[df[df['device_id'] == i].index[0],'longitude'], df.loc[df[df['device_id'] == j].index[0],'longitude']]
            lat = [df.loc[df[df['device_id'] == i].index[0],'latitude'], df.loc[df[df['device_id'] == j].index[0],'latitude']]
            distance = haversine(lon[0], lat[0], lon[1], lat[1])
            #print('i: %s j: %s :: %s %s %s' % (i,j, lon, lat, distance))
            df1.loc[i,j] = distance
            #try: df1.loc[i,j] = 1.0 / distance
            #except: 0
    df1 = df1 < 500    
    #df = df.values
    df1 = pd.DataFrame(np.array(df1.values, dtype=np.int16), index=df1.index, columns=df1.columns)
    # end distanceCube
    return(df1)

def haversine(lon1, lat1, lon2, lat2):
    """ Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r
# lon = [113.58, 103.8]
# lat = [23.55, 36.07]
# haversine(lon[0], lat[0], lon[1], lat[1])

def getVarName(level=-3):
    # source: https://stackoverflow.com/questions/2749796/how-to-get-the-original-variable-name-of-variable-passed-to-a-function
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[level]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    return(vars_name)

def printdf(df):
    res = getVarName(-3)
    try: df = df[df['longitude'] > 0]
    except: ''
    df = df.sort_values(by='device_id', ascending=True)
    with pd.option_context('display.max_rows', 400, 'display.max_columns', 4000, 'display.width', 1000000):
        print(res)
        print(df)
        print(df.shape)
        print()

In [ ]:
df0 = df00[df00['ts'] == 1462060638] # slice001
#for i in range(1,10):
#for i in range(lmin,lmax):
cnt = 0
def estimate_time_to_completion():
    ''
import time
tstart = time.time()
for i in df00_uniqueTimestamps.index:
    #print(i)
    #df01 = df00[df00['ts'] == (1462060638 + i)] # slice00%s % i
    #printdf(df0)
    #df0 = df0.combine_first(df01)
    #printdf(df00)
    #printdf(df01)
    #printdf(df0)
    #print('---')
    cnt += 1
    if cnt % 1000 == 0:
        #estimate_time_to_completion()
        tnow = time.time()
        tdiff = tnow - tstart
        estimateEndTime = tnow / cnt * 1
        #print('%s %s %s %s' %(cnt, i, tdiff, estimateEndTime))
    #if cnt > 300: break
    ''
dfCube = distanceCube(df0)
dfCube.to_csv('/kaggle/working/distanceCube.csv')
#printdf(dfCube)
with pd.option_context('display.max_rows', 400, 'display.max_columns', 4000, 'display.width', 1000000):
    print(dfCube)

In [ ]:
df = df0.sort_values('ts', ascending=True)

df = df.loc[:, 'device_id longitude latitude ts'.split(' ')]
# df = df.head(1000)
# dfg = df.groupby('ts')
# dfgd = dfg.describe()
# print(dfgd.sort_values(by=('longitude','count'), ascending=False))

distanceCube(df)

print()
print(df)
print(df.shape)
df = df.values
# df = df.reshape(20)
df = df[:,0:3].reshape(df.shape[0], df.shape[1]-1, 1)
# df = df.reshape(df.shape[0], df.shape[1], 1)

print(df.shape)
# df = df.tolist()
# import matplotlib.pylab as plt
# plt.plot(df[:,1], df[:,2])

In [ ]:
# for rotatable 3d plots
!pip install ipympl
# If using JupyterLab 2
!pip install nodejs
!jupyter labextension install @jupyter-widgets/jupyterlab-manager
!jupyter labextension install jupyter-matplotlib

In [ ]:
dir(dt.datetime)
dir(dt.datetime)
res = dt.datetime.strptime('2016-05-01 00:55:25', '%Y-%m-%d %H:%M:%S')
# dir(res)
res.timestamp()

In [ ]:
df_events = df_events.sort_values(by='device_id')[ (df_events['longitude'] > 0) | (df_events['latitude'] > 0) ].sample(n=100)

In [ ]:
df_events.shape

In [ ]:
dfg = df_events.groupby('device_id')
dfgd = dfg.describe()
dfgd

In [ ]:
dfgd.sort_values(by=('event_id','count'), ascending=False)

In [ ]:
dfgd

In [ ]:
df_events0 = df_events[ 
#     (df_events['device_id'] == '-1656894751624916732')
   (df_events['device_id'] == '1186608308763918427') 
   | (df_events['device_id'] == '3915082290673137129') 
] #.shape

In [ ]:
df = df_events0.loc[:, 'device_id timestamp longitude latitude'.split(' ')]
dfg = df.groupby('device_id')
dfgd = dfg.describe()
dfgd

### Plot a bunch of different maps showing the locations of the events

In [ ]:
# Set up plot
# df_events_sample = df_events.sample(n=100000)
df_events0 = df_events
df_events = df_events0
df_events_sample = df_events0
plt.figure(1, figsize=(96,6), dpi=200)

# Mercator of World
m1 = Basemap(projection='merc',
             llcrnrlat=-60,
             urcrnrlat=65,
             llcrnrlon=-180,
             urcrnrlon=180,
             lat_ts=0,
             resolution='c')

m1.fillcontinents(color='#191919',lake_color='#000000') # dark grey land, black lakes
m1.drawmapboundary(fill_color='#000000')                # black background
m1.drawcountries(linewidth=0.1, color="w")              # thin white line for country borders

# Plot the data
mxy = m1(df_events_sample["longitude"].tolist(), df_events_sample["latitude"].tolist())
m1.scatter(mxy[0], mxy[1], s=3, c="#1292db", lw=0, alpha=1, zorder=5)

plt.title("Global view of events")
plt.show()

Not surprisingly, most of the events are geo-located in China. We also see a few sporadic ones around the globe, most which look real (e.g., Austalia-Sydney/Melbourne/Perth).

One interesting thing is there are a number of events at (lat,lon) = (0,0), and also around that area where there is not much land.

In [ ]:
df_at0 = df_events[(df_events["longitude"]==0) & (df_events["latitude"]==0)]
df_near0 = df_events[(df_events["longitude"]>-1) &\
                     (df_events["longitude"]<1) &\
                     (df_events["latitude"]>-1) &\
                     (df_events["latitude"]<1)]

print("# events:", len(df_events))
print("# at (0,0)", len(df_at0))
print("# near (0,0)", len(df_near0))

Plot another mercator plot, but zooming in on China

In [ ]:
# Sample it down to only the China region
lon_min, lon_max = 75, 135
lat_min, lat_max = 15, 55

idx_china = (df_events["longitude"]>lon_min) &\
            (df_events["longitude"]<lon_max) &\
            (df_events["latitude"]>lat_min) &\
            (df_events["latitude"]<lat_max)

df_events_china = df_events[idx_china].sample(n=100000)

# Mercator of China
plt.figure(2, figsize=(96,6), dpi=300)

m2 = Basemap(projection='merc',
             llcrnrlat=lat_min,
             urcrnrlat=lat_max,
             llcrnrlon=lon_min,
             urcrnrlon=lon_max,
             lat_ts=35,
             #resolution='i'
             resolution='c'
            )

m2.fillcontinents(color='#191919',lake_color='#000000') # dark grey land, black lakes
m2.drawmapboundary(fill_color='#000000')                # black background
m2.drawcountries(linewidth=0.1, color="w")              # thin white line for country borders

# Plot the data
mxy = m2(df_events_china["longitude"].tolist(), df_events_china["latitude"].tolist())
m2.scatter(mxy[0], mxy[1], s=5, c="#1292db", lw=0, alpha=0.05, zorder=5)

plt.title("China view of events")
plt.show()

This is more interesting to visualise, and roughly matches what you'd expect from the population density in China, see e.g.

http://www.china-food-security.org/images/maps/pop/pop_2_h.jpg

In [ ]:
# Sample it down to only the Beijing region
lon_min, lon_max = 116, 117
lat_min, lat_max = 39.75, 40.25
lon_avg = float(lon_min + lon_max) / 2
lat_avg = float(lat_min + lat_max) / 2

idx_beijing = (df_events["longitude"]>lon_min) &\
              (df_events["longitude"]<lon_max) &\
              (df_events["latitude"]>lat_min) &\
              (df_events["latitude"]<lat_max)

df_events_beijing = df_events[idx_beijing]

# Mercator of Beijing
plt.figure(3, figsize=(24,6), dpi=89)

m3 = Basemap(projection='merc',
             llcrnrlat=lat_min,
             urcrnrlat=lat_max,
             llcrnrlon=lon_min,
             urcrnrlon=lon_max,
             lat_ts=35,
             resolution='c')

m3.fillcontinents(color='#191919',lake_color='#000000') # dark grey land, black lakes
m3.drawmapboundary(fill_color='#000000')                # black background
m3.drawcountries(linewidth=0.1, color="w")              # thin white line for country borders

# Plot the data
mxy = m3(df_events_beijing["longitude"].tolist(), df_events_beijing["latitude"].tolist())
m3.scatter(mxy[0], mxy[1], s=5, c="#1292db", lw=0, alpha=0.1, zorder=5)

plt.title("Beijing view of events")
plt.show()

import plotly.express as px
def visualize_trafic(df, center, zoom=9):
    fig = px.scatter_mapbox(df,
                            
                            # Here, plotly gets, (x,y) coordinates
                            lat="longitude",
                            lon="latitude",
                            
                            #Here, plotly detects color of series
                            color="device_id",
                            labels="device_id",
                            
                            zoom=zoom,
                            center=center,
                            height=600,
                            width=800)
    fig.update_layout(mapbox_style='stamen-terrain')
    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
    fig.update_layout(title_text="GPS trafic")
    fig.show()
center = {"lat":df_events_beijing['latitude'].mean(), "lon":df_events_beijing['longitude'].mean()}
# df_events_beijing.sample(n=100).shape
# visualize_trafic(df_events_beijing, center)
df_events_beijing

At this scale, you can actually see the finite resolution of the lat/lon values.

### Now we can try plotting the same for male/female, and show average age per grid

In [ ]:
# Load the train data and join on the events
fname = "/kaggle/input/china-mobile-user-gemographics/gender_age_train.csv"
df_train = pd.read_csv(fname, dtype={'device_id': np.str})

df_plot = pd.merge(df_train, df_events_beijing, on="device_id", how="inner")

df_m = df_plot[df_plot["gender"]=="M"]
df_f = df_plot[df_plot["gender"]=="F"]

In [ ]:
# Male/female plot
plt.figure(4, figsize=(12,12), dpi=144)


plt.subplot(121)
m4a = Basemap(projection='merc',
             llcrnrlat=lat_min,
             urcrnrlat=lat_max,
             llcrnrlon=lon_min,
             urcrnrlon=lon_max,
             lat_ts=35,
             resolution='c')
m4a.fillcontinents(color='#191919',lake_color='#000000') # dark grey land, black lakes
m4a.drawmapboundary(fill_color='#000000')                # black background
m4a.drawcountries(linewidth=0.1, color="w")              # thin white line for country borders
mxy = m4a(df_m["longitude"].tolist(), df_m["latitude"].tolist())
m4a.scatter(mxy[0], mxy[1], s=5, c="#1292db", lw=0, alpha=0.1, zorder=5)
plt.title("Male events in Beijing")

plt.subplot(122)
m4b = Basemap(projection='merc',
             llcrnrlat=lat_min,
             urcrnrlat=lat_max,
             llcrnrlon=lon_min,
             urcrnrlon=lon_max,
             lat_ts=35,
             resolution='c')
m4b.fillcontinents(color='#191919',lake_color='#000000') # dark grey land, black lakes
m4b.drawmapboundary(fill_color='#000000')                # black background
m4b.drawcountries(linewidth=0.1, color="w")              # thin white line for country borders
mxy = m4b(df_f["longitude"].tolist(), df_f["latitude"].tolist())
m4b.scatter(mxy[0], mxy[1], s=5, c="#fd3096", lw=0, alpha=0.1, zorder=5)
plt.title("Female events in Beijing")

plt.show()

Clearly some different patterns emerging, though it's not immediately clear whether or not this is just because there are different amounts of data for M/F.

In [ ]:
print("# M obs:", len(df_m))
print("# F obs:", len(df_f))

In [ ]:
# Make a pivot table showing average age per area of a grid, also store the counts
df_plot["lon_round"] = df_plot["longitude"].round(decimals=2)
df_plot["lat_round"] = df_plot["latitude"].round(decimals=2)

df_age = pd.pivot_table(df_plot,\
                        values="age",\
                        index="lon_round",\
                        columns="lat_round",\
                        aggfunc=np.mean)

df_cnt = pd.pivot_table(df_plot,\
                        values="age",\
                        index="lon_round",\
                        columns="lat_round",\
                        aggfunc="count")

In [ ]:
# Age plot
plt.figure(5, figsize=(24,12), dpi=55)

# Plot avg age per grid
plt.subplot(121)
m5a = Basemap(projection='merc',
             llcrnrlat=lat_min,
             urcrnrlat=lat_max,
             llcrnrlon=lon_min,
             urcrnrlon=lon_max,
             lat_ts=35,
             resolution='c')      
# Construct a heatmap
lons = df_age.index.values
lats = df_age.columns.values
x, y = np.meshgrid(lons, lats) 
px, py = m5a(x, y) 
data_values = df_age.values
masked_data = np.ma.masked_invalid(data_values.T)
cmap = plt.cm.viridis
cmap.set_bad(color="#191919")
# Plot the heatmap
m5a.pcolormesh(px, py, masked_data, cmap=cmap, zorder=5)
m5a.colorbar().set_label("average age")
plt.title("Average age per grid area in Beijing")

# Plot count per grid
plt.subplot(122)
m5b = Basemap(projection='merc',
             llcrnrlat=lat_min,
             urcrnrlat=lat_max,
             llcrnrlon=lon_min,
             urcrnrlon=lon_max,
             lat_ts=35,
             resolution='c')      
# Construct a heatmap 
data_values = df_cnt.values
masked_data = np.ma.masked_invalid(data_values.T)
cmap = plt.cm.viridis
cmap.set_bad(color="#191919")
# Plot the heatmap
m5b.pcolormesh(px, py, masked_data, cmap=cmap, zorder=5)
m5b.colorbar().set_label("count")
plt.title("Event count per grid area in Beijing")

plt.show()

Again, clearly some potentially interesting patterns.